مندرجہ ذیل نوٹ بکس چلانے کے لیے، اگر آپ نے ابھی تک نہیں کیا ہے، تو آپ کو .env فائل کے اندر openai کی کو `OPENAI_API_KEY` کے طور پر سیٹ کرنا ہوگا۔


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

اب ہم ایمبیڈنگ انڈیکس کو ایک پانڈاز ڈیٹافریم میں لوڈ کرنے جا رہے ہیں۔ ایمبیڈنگ انڈیکس ایک JSON فائل میں محفوظ ہے جس کا نام `embedding_index_3m.json` ہے۔ ایمبیڈنگ انڈیکس میں ہر یوٹیوب ٹرانسکرپٹ کے ایمبیڈنگز شامل ہیں، جو اکتوبر 2023 کے آخر تک کے ہیں۔


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

اب ہم ایک فنکشن بنائیں گے جس کا نام `get_videos` ہوگا، جو کہ Embedding Index میں دی گئی تلاش کے لیے ویڈیوز تلاش کرے گا۔ یہ فنکشن وہ پانچ ویڈیوز واپس کرے گا جو تلاش سے سب سے زیادہ ملتی جلتی ہوں گی۔ یہ فنکشن اس طرح کام کرتا ہے:

1. سب سے پہلے، Embedding Index کی ایک نقل تیار کی جاتی ہے۔
2. اس کے بعد، تلاش کے لیے Embedding کو OpenAI Embedding API کی مدد سے نکالا جاتا ہے۔
3. پھر Embedding Index میں ایک نیا کالم بنایا جاتا ہے جس کا نام `similarity` ہے۔ اس کالم میں تلاش کے Embedding اور ہر ویڈیو سیگمنٹ کے Embedding کے درمیان cosine similarity رکھی جاتی ہے۔
4. اس کے بعد، Embedding Index کو `similarity` کالم کی بنیاد پر فلٹر کیا جاتا ہے۔ صرف وہی ویڈیوز رکھی جاتی ہیں جن کی cosine similarity کم از کم 0.75 ہو۔
5. آخر میں، Embedding Index کو `similarity` کالم کے مطابق ترتیب دیا جاتا ہے اور سب سے اوپر والی پانچ ویڈیوز واپس کی جاتی ہیں۔


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

یہ فنکشن بہت سادہ ہے، یہ صرف تلاش کی گئی عبارت کے نتائج ظاہر کرتا ہے۔


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. سب سے پہلے، ایمبیڈنگ انڈیکس کو ایک پانڈاز ڈیٹافریم میں لوڈ کیا جاتا ہے۔
2. اس کے بعد، صارف سے کہا جاتا ہے کہ وہ ایک سوال درج کرے۔
3. پھر `get_videos` فنکشن کو کال کیا جاتا ہے تاکہ ایمبیڈنگ انڈیکس میں اس سوال کے لیے تلاش کی جا سکے۔
4. آخر میں، `display_results` فنکشن کو کال کیا جاتا ہے تاکہ نتائج صارف کو دکھائے جا سکیں۔
5. پھر صارف سے کہا جاتا ہے کہ وہ ایک اور سوال درج کرے۔ یہ عمل اس وقت تک جاری رہتا ہے جب تک صارف `exit` نہیں لکھتا۔

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ur.png)

آپ سے سوال درج کرنے کے لیے کہا جائے گا۔ سوال لکھیں اور انٹر دبائیں۔ ایپلیکیشن آپ کو ان ویڈیوز کی فہرست دے گی جو آپ کے سوال سے متعلق ہیں۔ اس کے علاوہ، ایپلیکیشن آپ کو اس ویڈیو کے اس حصے کا لنک بھی دے گی جہاں آپ کے سوال کا جواب موجود ہے۔

یہاں کچھ سوالات ہیں جنہیں آپ آزما سکتے ہیں:

- Azure Machine Learning کیا ہے؟
- کنوولوشنل نیورل نیٹ ورکس کیسے کام کرتے ہیں؟
- نیورل نیٹ ورک کیا ہے؟
- کیا میں Jupyter Notebooks کو Azure Machine Learning کے ساتھ استعمال کر سکتا ہوں؟
- ONNX کیا ہے؟


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**اعلانِ دستبرداری**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کی پوری کوشش کرتے ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہے۔ اصل دستاویز کو اس کی اصل زبان میں مستند ماخذ سمجھا جانا چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ تجویز کیا جاتا ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر نہیں ہوگی۔
